In [1]:
from langchain.agents import create_openai_tools_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain.tools import StructuredTool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import os

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Set the environment variable for the API key:
os.environ['SERPAPI_API_KEY'] = 'INSERT_API_KEY_HERE'

In [2]:
# Define Tools:
def google_search(query: str):
    """Tool to search Google"""
    return SerpAPIWrapper().run(query)

def save_interview(raw_interview_text: str):
    """Tool to save the interview. You must pass the entire interview and conversation in here.
    The interview will then be saved to a local file. Remember to include all of the previous chat messages.
    Include all of the messages with the user and the AI, here is a good response:
    AI: some text
    Human: some text
    ...
    ---
    """
    # Save to local file:
    with open("interview.txt", "w") as f:
        f.write(raw_interview_text)
    return f"Interview saved! Content: {raw_interview_text}. File: interview.txt. You must tell the user that the interview is saved."

def get_pokemon(pokemon_name: str):
    """Tool to get a single Pokemon"""
    import requests
    clean_pokemon_name = pokemon_name.lower()
    # Get the Pokemon
    try:
        pokemon = requests.get(f"https://pokeapi.co/api/v2/pokemon/{clean_pokemon_name}").json()
        if pokemon:
            return f"Pokemon found! Name: {pokemon['name']}. Height: {pokemon['height']}. Weight: {pokemon['weight']}."
        else:
            return "Pokemon not found!"
    except Exception as e:
        print(e)
        return "Pokemon not found! Try a different pokemon."

In [3]:
# Convert functions to Structured Tools:
google_search = StructuredTool.from_function(google_search)
save_interview = StructuredTool.from_function(save_interview)
get_single_pokemon = StructuredTool.from_function(get_pokemon)

In [4]:
from pydantic.v1 import BaseModel
from typing import Optional, Type, Union, Literal, List
from langchain.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools.base import ToolException
from langchain_core.documents import Document
import tempfile
import requests

class ArgumentType(BaseModel):
    url: str
    file_type: Union[Literal["pdf"], Literal["txt"]]

class SummarizeFileFromURL(BaseTool):
    name: str = "SummarizeFileFromURL"
    description: str = "Summarize a file from a URL."
    args_schema: Type[ArgumentType] = ArgumentType

    def _run(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        from langchain_community.document_loaders import PyPDFLoader, TextLoader
        from langchain.chains.summarize import load_summarize_chain

        response = requests.get(url)
        if response.status_code != 200:
            raise ToolException("The URL is not valid. Please try another URL.")

        if file_type == "pdf":
            with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as tmp:
                tmp.write(response.content)
                pdf_loader = PyPDFLoader(file_path=tmp.name)
                documents: List[Document] = pdf_loader.load()

        else:
            with tempfile.NamedTemporaryFile(suffix=".txt", delete=True, mode="w+t") as tmp:
                tmp.write(response.text)
                text_loader = TextLoader(tmp.name)
                documents: List[Document] = text_loader.load()

        chain = load_summarize_chain(llm=ChatOpenAI(), chain_type="map_reduce")
        return chain.run(documents)

    async def _arun(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This tool does not support async yet!")

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

file_summarizer = SummarizeFileFromURL()
file_summarizer.handle_tool_error = _handle_error

In [5]:
# Tools:
tools = [google_search, save_interview, get_single_pokemon, file_summarizer]

In [6]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [7]:
# Create the llm:
llm = ChatOpenAI(temperature=0)

# Create the ChatPromptTemplate:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the system message:
system_message = SystemMessage(
    content="""Act as a helpful AI interview assistant,
               Always ask a follow-up question, especially after returning results.
               Ask one question at a time.
               Always be in interview mode or tool mode.
               Ask the user about the desired topic.
               """
)

# Initialize memory:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Create the Prompt:
prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="history"), # This is where the user will write/read their messages from as memory
        ("user", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # This is where the agent will write/read its messages from
    ]
)

agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

/var/folders/0k/3rxx7x4x7_7f028m32kgbqzh0000gn/T/ipykernel_95908/1333559002.py:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)


In [8]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# Formats the python function tools into JSON schema and binds them to the model:
llm_with_tools = llm.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


# Setting up the agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [9]:
# Create an agent executor by passing in the agent, tools and memory:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [10]:
agent_executor.invoke({"input": "My name is James, I'm a computer programmer and I like to play cosy games."})



> Entering new AgentExecutor chain...
Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?

> Finished chain.


{'input': "My name is James, I'm a computer programmer and I like to play cosy games.",
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={})],
 'output': "Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?"}

In [11]:
agent_executor.invoke({"input": "What is my name and my profession?"})



> Entering new AgentExecutor chain...
Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?

> Finished chain.


{'input': 'What is my name and my profession?',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name and my profession?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?", additional_kwargs={}, response_metadata={})],
 'output': "Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize

In [12]:
agent_executor.invoke({"input": "Get information about Pikachu"})



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Pikachu'}`


Pokemon found! Name: pikachu. Height: 4. Weight: 60.Pikachu is a popular Pokémon! It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?

> Finished chain.


{'input': 'Get information about Pikachu',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name and my profession?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Get information about Pikachu', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Pikachu is a popular Pokémon! It has a height of 4 and a weight of 60. Is th

In [13]:
agent_executor.invoke({"input": "Get information about Charmander"})



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Charmander'}`


Pokemon found! Name: charmander. Height: 6. Weight: 85.Charmander is another iconic Pokémon! It has a height of 6 and a weight of 85. If you have any more Pokémon you'd like to learn about or any other questions, feel free to ask!

> Finished chain.


{'input': 'Get information about Charmander',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name and my profession?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Get information about Pikachu', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Pikachu is a popular Pokémon! It has a height of 4 and a weight of 60. Is

In [14]:
agent_executor.invoke(
    {
        "input": "I want a comprehensive summary of our conversation! Please can you save it?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `save_interview` with `{'raw_interview_text': "User: My name is James, I'm a computer programmer and I like to play cosy games.\nAssistant: Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?\nUser: What is my name and my profession?\nAssistant: Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?\nUser: Get information about Pikachu\nAssistant: Pikachu is a popular Pokémon! It has a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu or any other Pokémon?\nUser: Get information about Charmander\nAssistant: Charmander is another iconic Pokémon! It has a height of 6 and a weight of 85. If you have any more Pokémon you'd like to learn about or 

{'input': 'I want a comprehensive summary of our conversation! Please can you save it?',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name and my profession?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Get information about Pikachu', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Pikachu is a popular Pokémon! 

In [15]:
try:
    agent_executor.invoke({"input":"I want to summarize this https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx"})
except Exception as e:
    error_message = str(e)
    chat = ChatOpenAI()
    clean_error_message = chat.invoke([SystemMessage(content=error_message + 'Clean the error message, this will be shown to a user.')])
    print(clean_error_message.content)



> Entering new AgentExecutor chain...

Invoking: `SummarizeFileFromURL` with `{'url': 'https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx', 'file_type': 'pdf'}`




invalid pdf header: b'PK\x03\x04\x14'
EOF marker not found


The stream has ended unexpectedly. Please try refreshing the page or coming back later. Thank you.


# This line will take several minutes as we are summarizing a 41 page .pdf file:
agent_executor.invoke(
    {
        "input": "I want to summarize this https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf "
    }
)

In [16]:
# Trying to use the google search tool:
agent_executor.invoke(
    {
        "input": "I want to see the latest news about Tesla from Google, show me the latest 2 articles"
    }
)



> Entering new AgentExecutor chain...

Invoking: `google_search` with `{'query': 'latest news about Tesla'}`


[{'title': 'Tesla brings latest juicy offer to table to boost end of year EV sales', 'link': 'https://www.teslarati.com/tesla-brings-latest-juicy-offer-to-table-to-boost-end-of-year-ev-sales/', 'source': 'Teslarati', 'date': '2 hours ago', 'thumbnail': 'https://serpapi.com/searches/6737747ea869215b14560902/images/ea24528a427ea7d964d707410ce1a6aa25fd468b2acd103d.jpeg'}, {'title': 'Tesla’s Latest Supercharger Update Brings Big Speed Improvements', 'link': 'https://insideevs.com/news/741083/tesla-v4-supercharger-cabinet-speed/', 'source': 'InsideEVs', 'date': '8 hours ago', 'thumbnail': 'https://serpapi.com/searches/6737747ea869215b14560902/images/ea24528a427ea7d9bf08356f4eee71615fb9389967ccf5a3.jpeg'}, {'title': 'Tesla stock to settle after recent surge: Nuveen CIO', 'link': 'https://finance.yahoo.com/news/tesla-stock-to-settle-after-recent-surge-nuveen-cio-204349139.html', 's

{'input': 'I want to see the latest news about Tesla from Google, show me the latest 2 articles',
 'history': [HumanMessage(content="My name is James, I'm a computer programmer and I like to play cosy games.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, James! It's great to hear that you enjoy playing cosy games. Is there a specific cosy game that you're currently playing or interested in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name and my profession?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Your name is James, and you mentioned that you are a computer programmer. It's nice to know more about you! Is there a particular aspect of computer programming that you specialize in or enjoy the most?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Get information about Pikachu', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Pikachu is a popular 